In [1]:
import sys
sys.path.append('../')
from rocky.core import rocky
from rocky.triangle import Triangle

In [2]:
t = Triangle.from_taylor_ashe()
t.df

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Create a Rocky object
rky = rocky()

# add triangle
# rocky_inputs_file = r"O:\PARM\Individual folders\A Weaver\Reserving\rocky\rocky-inputs.xlsx"
# rky.FromExcel(filename=rocky_inputs_file, sheet_name='triangle', id='paid_loss')
rky.SampleTri('taylor_ashe')

# add model
rky.AddModel('icrfs', model_class='loglinear', tri=rky.paid_loss, use_cal=True, plot_width=None)

# set hyperparameters
rky.icrfs.SetHyperparameters(alpha=0.05, l1_ratio=0.1)

# fit model
rky.icrfs.Fit()

# basic hetero adjustment (one-step, no cross-validation,
# just one parameter per development period)
rky.icrfs.FitHetero()

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.
Fitting hetero adjustment: (Step/RMSE/L2-Norm)
1/0.0068 

In [4]:
import numpy as np
import pandas as pd

def lasso_path(X, y, alphas=None, **kwargs):
    """Get the Lasso path (weights at which each feature is removed)."""
    from sklearn.linear_model import Lasso
    from tqdm import tqdm
    
    # If no alphas are provided, create a starter alphpa
    if alphas is None:
        alpha = 0

    # Initialize dictionary to store results
    coef_dict = {}

    # For each alpha value in the list...
    while True:
        # Create a Lasso model with that alpha
        lasso = Lasso(alpha=alpha, **kwargs)
        
        # Fit the model
        lasso.fit(X, y)

        # Store the coefficients
        coef_dict[alpha] = lasso.coef_
        
        # increment alpha
        alpha += 0.0025

        # If all coefficients are 0, stop
        if len(list(coef_dict.keys())) == X.columns.shape[0]:
            break
    
    # Convert the dictionary into a DataFrame
    coef_df = pd.DataFrame(coef_dict, index=X.columns).T

    # Convert DataFrame to a format with feature, alpha and coef
    coef_df = coef_df.unstack().reset_index()
    coef_df.columns = ['Feature', 'Alpha', 'Coef']
    
    # Filter out rows where Coef is 0
    coef_df = coef_df[coef_df['Coef'] != 0]
    
    # Sort DataFrame by Alpha descending
    coef_df.sort_values('Alpha', ascending=False, inplace=True)
    
    # Get the last non-zero coef for each feature
    coef_df = coef_df.groupby('Feature').first().reset_index()
    
    return coef_df.sort_values('Alpha', ascending=False)


In [5]:
X = rky.icrfs.GetX('train')
y = rky.icrfs.GetY('train')
df = lasso_path(X, y)
df

,Feature,Alpha,Coef
22,development_period_006,0.0900,-0.314474
21,development_period_005,0.0900,-0.185221
18,development_period_002,0.0900,1.371040
23,development_period_007,0.0650,-0.004100
24,development_period_008,0.0400,-0.021180
12,calendar_period_005,0.0250,0.017292
0,accident_period_2002,0.0225,0.027481
16,calendar_period_009,0.0225,0.009658
26,development_period_010,0.0225,-0.062660
15,calendar_period_008,0.0125,-0.028204


In [7]:
rky.icrfs.TrendPlot('dev')

Development Period
Development Period
